In [3]:
import pandas as pd
import numpy as np
import csv
import re
from langdetect import detect
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

In [4]:
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+' # For removing mentions.
pat2 = r'https?://[^ ]+' # For removing links.
combined_pat = r'|'.join((pat1, pat2)) # For removing links.
www_pat = r'www.[^ ]+' # For removing links.
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"} # For removing negations.
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b') # For removing negations.

def tweet_cleaner(text):
      """ This cleans the tweets text content
          so it can be machine readable.
      """
      soup = BeautifulSoup(text, 'lxml') # Decodes HTML.
      souped = soup.get_text()
      try: # UTF decoding.
          bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
      except:
          bom_removed = souped
      stripped = re.sub(combined_pat, '', bom_removed) # Gets rid of links.
      stripped = re.sub(www_pat, '', stripped) # Gets rid of links.
      lower_case = stripped.lower()
      neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
      letters_only = re.sub("[^a-zA-Z]", " ", neg_handled) # Only lets letters in.
      words = [x for x  in tok.tokenize(letters_only) if len(x) > 1] # Below removes whitespace
      return (" ".join(words)).strip()

In [6]:
df = pd.read_csv('teslaFullDataset.csv', engine='python') #read in csv

In [25]:
length_of_df = len(df)
print(length_of_df)
df.head()

767393


,Like count,Retweet count,date,tweet
0,1.0,0.0,2019-02-28 23:40:44,with many stores closing going to be harder to...
1,0.0,0.0,2019-02-28 23:38:48,tesla drives down price for model to
2,0.0,0.0,2019-02-28 23:50:31,feel like everyone is talking about the car bu...
3,0.0,0.0,2019-02-28 23:54:06,pixie is taking liking to tesla dogslife trave...
4,0.0,0.0,2019-02-28 23:52:55,and tesla hardworking brilliant workers


In [41]:
tic = time.perf_counter()

for index, row in df.iterrows():
    df['tweet'][row] = tweet_cleaner(df["tweet"][row]) #why tf doesn't this work 
    
toc = time.perf_counter()
print(f"Cleaned {len(df)} in {toc - tic:0.4f} seconds")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [42]:
df[4000:4010] #ceck random range of dataset to ensure it was properly cleaned

,Like count,Retweet count,date,tweet
4000,23.0,1.0,2019-02-25 11:05:09,#Tesla Model S and X will now come standard wi...
4001,11.0,2.0,2019-02-25 10:30:07,Tesla aumenta las entregas del Model 3 en Chin...
4002,12.0,6.0,2019-02-25 10:53:43,Titulares de Lunes 25 de Febrero de 2019 en No...
4003,0.0,0.0,2019-02-25 11:53:49,"#TESLA, un actor claro en el #MWC19 https://t...."
4004,0.0,0.0,2019-02-25 11:41:10,Alexandre Brasseur (Vintage Mecanic): «On a fa...
4005,0.0,1.0,2019-02-25 10:42:47,Alexandre Brasseur (Vintage Mecanic): «On a fa...
4006,26.0,1.0,2019-02-25 12:09:09,Looks like this Model 3 is getting checked out...
4007,7.0,3.0,2019-02-25 11:24:59,The #Tesla #Model3 is the number 🥇selling EV ⚡...
4008,0.0,0.0,2019-02-25 10:41:20,Alexandre Brasseur (Vintage Mecanic): «On a fa...
4009,0.0,0.0,2019-02-25 11:15:10,Los vehículos 🚗 más deseados en el mundo 🌎 bas...


In [17]:
#df.drop('Source', inplace=True, axis=1)
#df.drop('Unnamed: 0', inplace=True, axis=1)
#df.drop('tweet_id', inplace=True, axis=1)
#df.drop('User location', inplace = True, axis = 1) #delete columns we don't need

In [ ]:
type(df)

In [ ]:
j = len(df)
i = 0 #remove 
for index, row in df.iterrows():
    if detect(df["tweet"][row])!= 'en': #this should work?
        df.drop(index, inplace=True)
print(j - len(df), "rows removed") #print tumber of rows that got removed

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
def sentimentScore(Tweet):
    analyser = SentimentIntensityAnalyzer()
    score_values = []
    for tweet in df['tweet']:
        vs1 = analyzer.polarity_scores(tweet)
        score_values.append(vs1)
    return score_values

In [ ]:
sentiment_scores = pd.DataFrame(sentimentScore(df['tweet']))
df.head()